## BERTopic Model

### Import data

In [7]:
import os
import json
from gensim.test.utils import datapath

tmp = datapath(os.getcwd() + '\\tmp\\model.bertopicmodel')
dirpath = os.getcwd() + '\\preprocessing\\'

with open(dirpath+'raw_texts.json', 'r') as f:
    documents = json.load(f)
    texts = [' '.join(doc) for doc in documents]

# List of stopwords
with open(dirpath+'swl_optimum.txt', 'r', encoding='utf-8') as f:
    swl = f.read().split('\n')

In [8]:
import pandas as pd

# Convert to DataFrame
df = pd.DataFrame(texts, columns=['headline_text'])
df.head()

,headline_text
0,9 март 2016 год произойти полный солнечный зат...
1,28 сентябрь состояться полный лунный затмение ...
2,20 март 2015 год произойти полный солнечный за...
3,22 июль 2009 год произойти самый продолжительн...
4,первый упоминание о солнечный затмение относит...


### Create a custom embedding model

In [9]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

### Dimensionality reduction & clustering

In [10]:
from umap import UMAP
from hdbscan import HDBSCAN

umap_model = UMAP(n_neighbors=20, n_components=1, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=15, min_samples=3, \
                        gen_min_span_tree=True, prediction_data=True)

### Create a custom vectorizer model

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# Filter out tokens by frequency
vectorizer_model = CountVectorizer(stop_words=swl, ngram_range=(1,2), min_df=10, max_df=0.9) 

### Initialize and run the BERTopic model

In [12]:
from bertopic import BERTopic

bertopic_model = BERTopic(top_n_words=15, language='multilingual', \
                          embedding_model=embedding_model, umap_model=umap_model, \
                          hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model, \
                          calculate_probabilities=True, verbose=True)

topics, probs = bertopic_model.fit_transform(df.headline_text)

Batches: 100%|██████████| 40/40 [02:54<00:00,  4.36s/it]
2022-06-26 17:17:53,250 - BERTopic - Transformed documents to Embeddings
2022-06-26 17:18:01,542 - BERTopic - Reduced dimensionality
2022-06-26 17:18:01,701 - BERTopic - Clustered reduced embeddings


In [13]:
# Save the model
bertopic_model.save(tmp)

In [14]:
# Load the model
BERTopic.load(tmp)

### Reduce the number of topics (optional)

In [15]:
new_topics, new_probs = bertopic_model.reduce_topics(df.headline_text, topics, probs, \
                                                     nr_topics=15)

2022-06-26 17:18:19,863 - BERTopic - Reduced number of topics from 35 to 16


### Print the results

In [16]:
# -1 refers to all outliers and should typically be ignored
freq = bertopic_model.get_topic_info()
freq.head(10)

,Topic,Count,Name
0,0,129,0_магнитный_электрон_плёнка_решётка
1,1,110,1_ген_мутация_геном_самец
2,-1,104,-1_премия_звезда_губка_физика
3,2,98,2_ген_позвоночный_червь_нервный
4,3,95,3_звезда_солнечный_солнце_космический
5,4,84,4_физика_протон_электрон_фотон
6,5,83,5_птица_самка_самец_яйцо
7,6,81,6_кость_находка_лес_дерево
8,7,79,7_белок_вирус_днк_фермент
9,8,77,8_порода_кора_континентальный_океанический


In [17]:
bertopic_model.get_topics()

{-1: [('премия', 0.02749462117341433),
  ('звезда', 0.02707049803206763),
  ('губка', 0.025541898811620636),
  ('физика', 0.024826239867064753),
  ('вселенная', 0.02430223132404903),
  ('материя', 0.023670445742358572),
  ('нобелевский', 0.022557448952750293),
  ('космический', 0.02052370329860953),
  ('уравнение', 0.018677618750012),
  ('гравитационный', 0.018215310029201248),
  ('излучение', 0.018116034242933666),
  ('ген', 0.017905771726122326),
  ('нобелевский премия', 0.01765746003165501),
  ('белок', 0.017095238259628432),
  ('электрон', 0.016854367292880132)],
 0: [('магнитный', 0.07863650356469797),
  ('электрон', 0.07365082042807805),
  ('плёнка', 0.03827355333891728),
  ('решётка', 0.03392410626732158),
  ('водород', 0.03358481621807547),
  ('излучение', 0.03310317568962922),
  ('металл', 0.03250351849331972),
  ('заряд', 0.03242028020813716),
  ('луч', 0.031883181325749965),
  ('физика', 0.025890400286888462),
  ('кристалл', 0.021777514087509445),
  ('сопротивление', 0.02111

In [18]:
# Select the most frequent topic
bertopic_model.get_topic(0)

[('магнитный', 0.07863650356469797),
 ('электрон', 0.07365082042807805),
 ('плёнка', 0.03827355333891728),
 ('решётка', 0.03392410626732158),
 ('водород', 0.03358481621807547),
 ('излучение', 0.03310317568962922),
 ('металл', 0.03250351849331972),
 ('заряд', 0.03242028020813716),
 ('луч', 0.031883181325749965),
 ('физика', 0.025890400286888462),
 ('кристалл', 0.021777514087509445),
 ('сопротивление', 0.021110515907865172),
 ('водородный', 0.020253526174527272),
 ('нм', 0.01960532851781696),
 ('днк', 0.019327201215474377)]